---
jupyter: python3
toc: true
title: Bigquery_Window 함수의 window_frame_clause 정리
date: 2023-01-12
categories: SQL
author: limyj0708
comments:
  giscus:
    repo: quarto-dev/quarto-docs
format:
    html:
        page-layout: full
highlight-style: github
---

> 정리해두지 않으면 항상 헷갈린다.


- 문법은 아래 설명이 너무 명확해서 더 볼 것이 없다.
  - [Link](https://cloud.google.com/bigquery/docs/reference/standard-sql/window-function-calls#def_window_frame)
- frame_start와 frame_end의 조합 제약사항들만 잘 살펴보면 된다. 

```
window_frame_clause:
  { rows_range } { frame_start | frame_between }

rows_range:
  { ROWS | RANGE }

frame_between:
  {
    BETWEEN  unbounded_preceding AND frame_end_a
    | BETWEEN numeric_preceding AND frame_end_a
    | BETWEEN current_row AND frame_end_b
    | BETWEEN numeric_following AND frame_end_c
  }

frame_start:
  { unbounded_preceding | numeric_preceding | [ current_row ] }

frame_end_a:
  { numeric_preceding | current_row | numeric_following | unbounded_following }

frame_end_b:
  { current_row | numeric_following | unbounded_following }

frame_end_c:
  { numeric_following | unbounded_following }

unbounded_preceding:
  UNBOUNDED PRECEDING

numeric_preceding:
  numeric_expression PRECEDING

unbounded_following:
  UNBOUNDED FOLLOWING

numeric_following:
  numeric_expression FOLLOWING

current_row:
  CURRENT ROW
```

## 1. ROWS
- 현재 행의 OFFSET 기반으로 윈도우 프레임을 정의함.
  - OFFSET 기반이므로, 현재 행은 index 0. 범위 2을 잡으면, 0~2이므로, 현재 행을 포함하여 행 3개를 프레임으로 잡게 됨

### 1-1. UNBOUNDED PRECEDING
  - PARTITION BY의 시작 부분을 참조
  - 즉, `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW` 라면, 윈도우 프레임의 범위는 PARTITION BY의 시작부터 현재 행까지임
  - 예시 쿼리

```SQL
SELECT
    employee_number
  , last_name
  , first_name
  , salary
  , dept_id
  , SUM(salary) OVER(PARTITION BY dept_id
                     ORDER BY salary
                     ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS sum_salary
FROM `bigquery-personal.test_ground.test_emp`
```

- 결과
  - PARTITON BY dept_id의 시작부터 현재 행까지 누적으로 SUM(salary)가 되고 있음을 알 수 있다.
  
| 행 |  employee_number |  last_name |  first_name |  salary |  dept_id |  sum_salary |
|----|------------------|------------|-------------|---------|----------|-------------|
|  1 |             1004 | Horvath    | Jack        |   42000 |      501 |       42000 |
|  2 |             1003 | Everest    | Brad        |   71000 |      501 |      113000 |
|  3 |             1005 | Kate       | Smith       |   72000 |      501 |      185000 |
|  4 |             1006 | blank      | Pencil      |   80000 |      501 |      265000 |
|  5 |             1007 | mobile     | phone       |  100000 |      501 |      365000 |
|  6 |             1002 | Anderson   | Jane        |   57500 |      500 |       57500 |
|  7 |             1008 | Lim        | Smith       |   58000 |      500 |      115500 |
|  8 |             1011 | Johnson    | Sally       |   58000 |      500 |      173500 |
|  9 |             1001 | Smith      | John        |   62000 |      500 |      235500 |
| 10 |             1010 | Nakamura   | Shin        |   62000 |      500 |      297500 |
| 11 |             1009 | Waterman   | Pencil      |   80000 |      500 |      377500 |
| 12 |             1012 | Johnson    | Emily       |   80000 |      500 |      457500 |

### 1-2. UNBOUNDED FOLLOWING
- PARTITION BY의 끝 부분을 참조
- 즉, `ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING` 이라면, 윈도우 프레임의 범위는 현재 행부터 PARITION BY의 끝까지임
- 예시 쿼리 

```SQL
SELECT
    employee_number
  , last_name
  , first_name
  , salary
  , dept_id
  , SUM(salary) OVER(PARTITION BY dept_id
                     ORDER BY salary
                     ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS sum_salary
FROM `bigquery-personal.test_ground.test_emp`
```

- 결과
  - 현재 행부터 PARTITON BY dept_id의 끝까지 누적으로 SUM(salary)가 되고 있음을 알 수 있다.
  
| 행 |  employee_number |  last_name |  first_name |  salary |  dept_id |  sum_salary |
|----|------------------|------------|-------------|---------|----------|-------------|
|  1 |             1002 | Anderson   | Jane        |   57500 |      500 |      457500 |
|  2 |             1008 | Lim        | Smith       |   58000 |      500 |      400000 |
|  3 |             1011 | Johnson    | Sally       |   58000 |      500 |      342000 |
|  4 |             1001 | Smith      | John        |   62000 |      500 |      284000 |
|  5 |             1010 | Nakamura   | Shin        |   62000 |      500 |      222000 |
|  6 |             1009 | Waterman   | Pencil      |   80000 |      500 |      160000 |
|  7 |             1012 | Johnson    | Emily       |   80000 |      500 |       80000 |
|  8 |             1004 | Horvath    | Jack        |   42000 |      501 |      365000 |
|  9 |             1003 | Everest    | Brad        |   71000 |      501 |      323000 |
| 10 |             1005 | Kate       | Smith       |   72000 |      501 |      252000 |
| 11 |             1006 | blank      | Pencil      |   80000 |      501 |      180000 |
| 12 |             1007 | mobile     | phone       |  100000 |      501 |      100000 |

## 1-3. NUMERIC PRECEDING, NUMERIC FOLLOWING
- NUMERIC PRECEDING : 현재 행부터 N만큼 위에 있는 행을 참조
- NUMERIC FOLLOWING : 현재 행부터 N만큼 아래에 있는 행을 참조
- 예시 쿼리

```SQL
SELECT
    employee_number
  , last_name
  , first_name
  , salary
  , dept_id
  , SUM(salary) OVER(PARTITION BY dept_id
                     ORDER BY salary
                     ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS sum_salary
FROM `bigquery-personal.test_ground.test_emp`
```

- 결과
  - 윈도우 프레임의 범위 : 현재 행보다 1행 위에 있는 행 ~ 현재 행 ~ 현재 행보다 1행 아래에 있는 행
  - 1행이면 1,2행 / 2행이면 1,2,3행의 값이 더해지게 된다.
  - 적절히 숫자를 바꾸면 된다.

| 행 |  employee_number |  last_name |  first_name |  salary |  dept_id |  sum_salary |
|----|------------------|------------|-------------|---------|----------|-------------|
|  1 |             1004 | Horvath    | Jack        |   42000 |      501 |       42000 |
|  2 |             1003 | Everest    | Brad        |   71000 |      501 |      113000 |
|  3 |             1005 | Kate       | Smith       |   72000 |      501 |      185000 |
|  4 |             1006 | blank      | Pencil      |   80000 |      501 |      223000 |
|  5 |             1007 | mobile     | phone       |  100000 |      501 |      252000 |
|  6 |             1002 | Anderson   | Jane        |   57500 |      500 |       57500 |
|  7 |             1011 | Johnson    | Sally       |   58000 |      500 |      115500 |
|  8 |             1008 | Lim        | Smith       |   58000 |      500 |      173500 |
|  9 |             1010 | Nakamura   | Shin        |   62000 |      500 |      178000 |
| 10 |             1001 | Smith      | John        |   62000 |      500 |      182000 |
| 11 |             1009 | Waterman   | Pencil      |   80000 |      500 |      204000 |
| 12 |             1012 | Johnson    | Emily       |   80000 |      500 |      222000 |

## 2. RANGE
- ORDER BY {sum_value}로 정렬했을 때, 같은 {sum_value}를 가지는 행들을 논리적으로 같은 행 범위라고 정의한다.

### 2-1. UNBOUNDED PRECEDING
- 예시 코드

```SQL
SELECT
    employee_number
  , last_name
  , first_name
  , salary
  , dept_id
  , SUM(salary) OVER(PARTITION BY dept_id
                  ORDER BY salary
                  RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS sum_salary
FROM `bigquery-personal.test_ground.test_emp`
```

- 결과
  - ORDER BY salary 이므로, 같은 salary인 행들은 전체가 한 번에 더해진다.
  - salary가 80000인 행이 행 6,7인데, 160000이 한 번에 더해지는 것을 볼 수 있다.
  
| 행 |  employee_number |  last_name |  first_name |  salary |  dept_id |  sum_salary |
|----|------------------|------------|-------------|---------|----------|-------------|
|  1 |             1002 | Anderson   | Jane        |   57500 |      500 |       57500 |
|  2 |             1008 | Lim        | Smith       |   58000 |      500 |      173500 |
|  3 |             1011 | Johnson    | Sally       |   58000 |      500 |      173500 |
|  4 |             1001 | Smith      | John        |   62000 |      500 |      297500 |
|  5 |             1010 | Nakamura   | Shin        |   62000 |      500 |      297500 |
|  6 |             1009 | Waterman   | Pencil      |   80000 |      500 |      457500 |
|  7 |             1012 | Johnson    | Emily       |   80000 |      500 |      457500 |
|  8 |             1004 | Horvath    | Jack        |   42000 |      501 |       42000 |
|  9 |             1003 | Everest    | Brad        |   71000 |      501 |      113000 |
| 10 |             1005 | Kate       | Smith       |   72000 |      501 |      185000 |
| 11 |             1006 | blank      | Pencil      |   80000 |      501 |      265000 |
| 12 |             1007 | mobile     | phone       |  100000 |      501 |       36500 |